In [228]:
import pandas as pd
import numpy as np
import time
import xlrd

# `maritime_freight_unloaded.csv`

In [131]:
data = pd.read_csv('data/raw/maritime_freight_unloaded.csv')

print(f"Shape is: {data.shape}")
data.head()

Shape is: (6019, 11)


,country,country_code,series,seriescode,measure,period,D1,D2,D3,data,footnoteref
0,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2017SEP,NaN,NaN,NaN,662.0,NaN
1,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016DEC,NaN,NaN,NaN,628.0,NaN
2,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016SEP,NaN,NaN,NaN,584.0,NaN
3,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016OCT,NaN,NaN,NaN,748.0,NaN
4,Latvia,428,Transport - International maritime freight unl...,4802.0,(thousand metric tons monthly average or calen...,2016YEAR,NaN,NaN,NaN,575.0,NaN


In [132]:
data.drop(['D1', 'D2', 'D3', 'footnoteref', 'measure', 'seriescode'], axis=1, inplace=True)
data.head()

,country,country_code,series,period,data
0,Latvia,428,Transport - International maritime freight unl...,2017SEP,662.0
1,Latvia,428,Transport - International maritime freight unl...,2016DEC,628.0
2,Latvia,428,Transport - International maritime freight unl...,2016SEP,584.0
3,Latvia,428,Transport - International maritime freight unl...,2016OCT,748.0
4,Latvia,428,Transport - International maritime freight unl...,2016YEAR,575.0


In [133]:
data = data[~data['data'].isna()]
data.isna().sum()

country         0
country_code    0
series          0
period          0
data            0
dtype: int64

In [134]:
data = data[(data['period'].str.contains('YEAR') == False) & (data['period'].str.contains('Q1') == False) & (data['period'].str.contains('Q2') == False) & (data['period'].str.contains('Q3') == False) & (data['period'].str.contains('Q4') == False)]

print(f"Shape is: {data.shape}")
data.head()

Shape is: (5136, 5)


,country,country_code,series,period,data
0,Latvia,428,Transport - International maritime freight unl...,2017SEP,662.0
1,Latvia,428,Transport - International maritime freight unl...,2016DEC,628.0
2,Latvia,428,Transport - International maritime freight unl...,2016SEP,584.0
3,Latvia,428,Transport - International maritime freight unl...,2016OCT,748.0
5,Latvia,428,Transport - International maritime freight unl...,2016JAN,423.0


In [135]:
def convert_date(cell):
    cell = cell.strip()
    year = cell[:4]
    month = cell[4:]
    return '-'.join([year, month])

data['period'] = data['period'].apply(convert_date)
data["period"] = pd.to_datetime(data["period"], format="%Y-%b")

data.dtypes

country                 object
country_code            object
series                  object
period          datetime64[ns]
data                   float64
dtype: object

In [136]:
data.head()

,country,country_code,series,period,data
0,Latvia,428,Transport - International maritime freight unl...,2017-09-01,662.0
1,Latvia,428,Transport - International maritime freight unl...,2016-12-01,628.0
2,Latvia,428,Transport - International maritime freight unl...,2016-09-01,584.0
3,Latvia,428,Transport - International maritime freight unl...,2016-10-01,748.0
5,Latvia,428,Transport - International maritime freight unl...,2016-01-01,423.0


In [137]:
data['series'].value_counts(dropna=False)

Transport - International maritime freight unloaded    2567
Transport - International maritime freight loaded      2523
Transport - International maritime freight total         46
Name: series, dtype: int64

In [138]:
data.rename(columns={
    'series' : 'freight_type',
    'data' : 'value_thousand_metric_tons'
    }, inplace=True)

data['freight_type'] = data['freight_type'].map({
    'Transport - International maritime freight unloaded' : 'unloaded',
    'Transport - International maritime freight loaded' : 'loaded',
    'Transport - International maritime freight total' : 'total' 
})

data.head()

,country,country_code,freight_type,period,value_thousand_metric_tons
0,Latvia,428,unloaded,2017-09-01,662.0
1,Latvia,428,unloaded,2016-12-01,628.0
2,Latvia,428,unloaded,2016-09-01,584.0
3,Latvia,428,unloaded,2016-10-01,748.0
5,Latvia,428,unloaded,2016-01-01,423.0


In [139]:
data.to_csv('data/cleaned/maritime_freight.csv', index=False)

# `SYB65_123_202209_Total Imports Exports and Balance of Trade.csv`

In [140]:
data = pd.read_csv('data/raw/SYB65_123_202209_Total Imports Exports and Balance of Trade.csv')
data.columns = ['Region/Country/Area', 'Total imports, exports and balance of trade', 'Year', 'Series', 'System of trade', 'System of trade footnote', 'Value', 'Footnotes', 'Source']

data = data.drop(data.index[0])

print(f"Shape is: {data.shape}")
data.head()

Shape is: (5059, 9)


,Region/Country/Area,"Total imports, exports and balance of trade",Year,Series,System of trade,System of trade footnote,Value,Footnotes,Source
1,1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),NaN,NaN,"5,099,057",NaN,"United Nations Statistics Division, New York, ..."
2,1,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),NaN,NaN,"10,577,013",NaN,"United Nations Statistics Division, New York, ..."
3,1,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),NaN,NaN,"15,275,115",NaN,"United Nations Statistics Division, New York, ..."
4,1,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),NaN,NaN,"16,482,346",NaN,"United Nations Statistics Division, New York, ..."
5,1,"Total, all countries or areas",2019,Imports CIF (millions of US dollars),NaN,NaN,"18,955,484",NaN,"United Nations Statistics Division, New York, ..."


In [141]:
data.drop(['Region/Country/Area', 'System of trade', 'System of trade footnote', 'Footnotes', 'Source'], axis=1, inplace=True)

print(f"Shape is: {data.shape}")
data.head()

Shape is: (5059, 4)


,"Total imports, exports and balance of trade",Year,Series,Value
1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),"5,099,057"
2,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),"10,577,013"
3,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),"15,275,115"
4,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),"16,482,346"
5,"Total, all countries or areas",2019,Imports CIF (millions of US dollars),"18,955,484"


In [142]:
data['Series'].value_counts()

Balance imports/exports (millions of US dollars)    1687
Imports CIF (millions of US dollars)                1686
Exports FOB (millions of US dollars)                1686
Name: Series, dtype: int64

In [143]:
data.rename(columns={
    'Total imports, exports and balance of trade' : 'total_imports_exports_and_balance_trade',
    'Year' : 'year',
    'Series' : 'trade_type',
    'Value' : 'value_million_usd'
    }, inplace=True)

data['trade_type'] = data['trade_type'].map({
    'Balance imports/exports (millions of US dollars)' : 'balance_import_export',
    'Imports CIF (millions of US dollars)' : 'imports',
    'Exports FOB (millions of US dollars)' : 'exports' 
})

data.head()

,total_imports_exports_and_balance_trade,year,trade_type,value_million_usd
1,"Total, all countries or areas",1995,imports,"5,099,057"
2,"Total, all countries or areas",2005,imports,"10,577,013"
3,"Total, all countries or areas",2010,imports,"15,275,115"
4,"Total, all countries or areas",2015,imports,"16,482,346"
5,"Total, all countries or areas",2019,imports,"18,955,484"


In [144]:
data["year"] = pd.to_datetime(data["year"], format="%Y")
data["value_million_usd"] = data["value_million_usd"].str.replace(',', '').astype(int)

data.head()

,total_imports_exports_and_balance_trade,year,trade_type,value_million_usd
1,"Total, all countries or areas",1995-01-01,imports,5099057
2,"Total, all countries or areas",2005-01-01,imports,10577013
3,"Total, all countries or areas",2010-01-01,imports,15275115
4,"Total, all countries or areas",2015-01-01,imports,16482346
5,"Total, all countries or areas",2019-01-01,imports,18955484


In [145]:
data.to_csv('data/cleaned/import_export_balance_trade.csv', index=False)

# `UN_paper_pulp_import_export.csv`

In [160]:
data = pd.read_csv('data/raw/UN_paper_pulp_import_export.csv')

print(f"Shape is: {data.shape}")
data.head()

Shape is: (38183, 6)


,Country or Area,Element,Year,Unit,Value,Value Footnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,233.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,646.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,140.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,45.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,291.0,Im


In [161]:
data.columns = [col.replace('or', '').replace('  ', '_').replace(' ', '').lower() for col in data.columns]
data.head()

,country_area,element,year,unit,value,valuefootnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,233.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,646.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,140.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,45.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,291.0,Im


In [162]:
data.drop('valuefootnotes', axis=1, inplace=True)

data.dropna(axis=0, inplace=True)

data["year"] = pd.to_datetime(data["year"], format="%Y")
data['value'] = data['value'].astype(int)

In [165]:
data.to_csv('data/cleaned/UN_paper_pulp_import_export.csv', index=False)

# `UN_wood_pulp_import_export.csv`

In [166]:
data = pd.read_csv('data/raw/UN_wood_pulp_import_export.csv')

print(f"Shape is: {data.shape}")
data.head()

Shape is: (36816, 6)


,Country or Area,Element,Year,Unit,Value,Value Footnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,245.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,622.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,127.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,32.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,167.0,Im


In [167]:
data.columns = [col.replace('or', '').replace('  ', '_').lower() for col in data.columns]
data.head()

,country_area,element,year,unit,value,value footnotes
0,Afghanistan,Import Quantity,2020.0,tonnes,245.0,Im
1,Afghanistan,Import Quantity,2019.0,tonnes,622.0,Im
2,Afghanistan,Import Quantity,2018.0,tonnes,127.0,Im
3,Afghanistan,Import Quantity,2017.0,tonnes,32.0,Im
4,Afghanistan,Import Quantity,2016.0,tonnes,167.0,Im


In [170]:
data.drop('value footnotes', axis=1, inplace=True)

data.dropna(axis=0, inplace=True)

data["year"] = pd.to_datetime(data["year"], format="%Y")
data['value'] = data['value'].astype(int)

In [171]:
data.dtypes

country_area            object
element                 object
year            datetime64[ns]
unit                    object
value                    int64
dtype: object

In [172]:
data.to_csv('data/cleaned/UN_wood_pulp_import_export.csv', index=False)

# `PULP AND PAPER CAPACITY BY COUNTRY 2020-2025.xls`

In [297]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/PULP AND PAPER CAPACITY BY COUNTRY 2020-2025.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

wood_pulp_capacities = []
wood_pulp_market_pulps = []

for index in range(2, 9, 2):
        wood_pulp_capacities.append([sheet.cell_value(rowx=12, colx=index) for sheet in sheets])
        wood_pulp_market_pulps.append([sheet.cell_value(rowx=12, colx=index+1) for sheet in sheets])

paper_paper_boards = []
for index in range(4, 8):
        paper_paper_boards.append([sheet.cell_value(rowx=56, colx=index) for sheet in sheets])

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'wood_pulp_capacity_2020': wood_pulp_capacities[0],
    'wood_pulp_market_pulps_2020': wood_pulp_market_pulps[0],
    'paper_paperboard_capacity_2020' :paper_paper_boards[0],
    'wood_pulp_capacity_2021': wood_pulp_capacities[1],
    'wood_pulp_market_pulps_2021': wood_pulp_market_pulps[1],
    'paper_paperboard_capacity_2021' :paper_paper_boards[1],
    'wood_pulp_capacity_2022': wood_pulp_capacities[2],
    'wood_pulp_market_pulps_2022': wood_pulp_market_pulps[2],
    'paper_paperboard_capacity_2022' :paper_paper_boards[2],
    'wood_pulp_capacity_2023': wood_pulp_capacities[3],
    'wood_pulp_market_pulps_2023': wood_pulp_market_pulps[3],
    'paper_paperboard_capacity_2023' :paper_paper_boards[3],
})

print(f"Shape is: {data.shape}")
data.head()

Shape is: (28, 13)


,country,wood_pulp_capacity_2020,wood_pulp_market_pulps_2020,paper_paperboard_capacity_2020,wood_pulp_capacity_2021,wood_pulp_market_pulps_2021,paper_paperboard_capacity_2021,wood_pulp_capacity_2022,wood_pulp_market_pulps_2022,paper_paperboard_capacity_2022,wood_pulp_capacity_2023,wood_pulp_market_pulps_2023,paper_paperboard_capacity_2023
0,AUS,2988.0,24.0,3661.0,2903.0,24.0,3541.0,2903.0,24.0,3549.0,2903.0,24.0,3549.0
1,AUT,1826.0,500.0,5590.0,1830.0,510.0,5703.0,1840.0,520.0,5795.0,1840.0,520.0,5965.0
2,BLR,440.0,400.0,460.0,590.0,400.0,660.0,590.0,400.0,660.0,590.0,400.0,680.0
3,BEL,352.0,.,1628.0,352.0,.,1854.0,352.0,.,1872.0,352.0,.,1872.0
4,BRA,23850.0,19195.0,13460.0,24300.0,19195.0,14050.0,24770.0,19195.0,14530.0,26770.0,21195.0,14530.0


In [298]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

,country,wood_pulp_capacity_2020,wood_pulp_market_pulps_2020,paper_paperboard_capacity_2020,wood_pulp_capacity_2021,wood_pulp_market_pulps_2021,paper_paperboard_capacity_2021,wood_pulp_capacity_2022,wood_pulp_market_pulps_2022,paper_paperboard_capacity_2022,wood_pulp_capacity_2023,wood_pulp_market_pulps_2023,paper_paperboard_capacity_2023
0,AUS,2988.0,24.0,3661.0,2903.0,24.0,3541.0,2903.0,24.0,3549.0,2903.0,24.0,3549.0
1,AUT,1826.0,500.0,5590.0,1830.0,510.0,5703.0,1840.0,520.0,5795.0,1840.0,520.0,5965.0
2,BLR,440.0,400.0,460.0,590.0,400.0,660.0,590.0,400.0,660.0,590.0,400.0,680.0
3,BEL,352.0,NaN,1628.0,352.0,NaN,1854.0,352.0,NaN,1872.0,352.0,NaN,1872.0
4,BRA,23850.0,19195.0,13460.0,24300.0,19195.0,14050.0,24770.0,19195.0,14530.0,26770.0,21195.0,14530.0


In [299]:
data.to_csv('data/cleaned/PULP AND PAPER CAPACITY BY COUNTRY 2020-2025.csv', index=False)

# `PULP AND PAPER PRODUCTION BY COUNTRY 2020.xls`

In [300]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/PULP AND PAPER PRODUCTION BY COUNTRY 2020.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

wood_pulp_productions = [sheet.cell_value(rowx=12, colx=2) for sheet in sheets]
wood_pulp_productions_market_pulps = [sheet.cell_value(rowx=12, colx=3) for sheet in sheets]

paper_paper_boards = [sheet.cell_value(rowx=56, colx=2) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'wood_pulp_production_2020': wood_pulp_productions,
    'wood_pulp_market_pulps_2020': wood_pulp_productions_market_pulps,
    'paper_paperboard_production_2020' : paper_paper_boards
})

print(f"Shape is: {data.shape}")
data.head()

Shape is: (35, 4)


,country,wood_pulp_production_2020,wood_pulp_market_pulps_2020,paper_paperboard_production_2020
0,ARG,620.0,0.0,1618.0
1,AUS,1395.0,24.0,3061.0
2,AUT,1581.0,439.0,4719.0
3,BLR,292.0,248.0,223.0
4,BEL,350.0,.,1610.0


In [301]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

,country,wood_pulp_production_2020,wood_pulp_market_pulps_2020,paper_paperboard_production_2020
0,ARG,620.0,0.0,1618.0
1,AUS,1395.0,24.0,3061.0
2,AUT,1581.0,439.0,4719.0
3,BLR,292.0,248.0,223.0
4,BEL,350.0,NaN,1610.0


In [302]:
data.to_csv('data/cleaned/PULP AND PAPER PRODUCTION BY COUNTRY 2020.csv', index=False)

# `PULP_AND_PAPER_PRODUCTION_BY_COUNTRY_2021.xls`

In [303]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/PULP_AND_PAPER_PRODUCTION_BY_COUNTRY_2021.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

wood_pulp_productions = [sheet.cell_value(rowx=12, colx=2) for sheet in sheets]
wood_pulp_productions_market_pulps = [sheet.cell_value(rowx=12, colx=3) for sheet in sheets]

paper_paper_boards = [sheet.cell_value(rowx=56, colx=2) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'wood_pulp_production_2021': wood_pulp_productions,
    'wood_pulp_market_pulps_2021': wood_pulp_productions_market_pulps,
    'paper_paperboard_production_2021' : paper_paper_boards
})

print(f"Shape is: {data.shape}")
data.head()

Shape is: (31, 4)


,country,wood_pulp_production_2021,wood_pulp_market_pulps_2021,paper_paperboard_production_2021
0,ARG,609.0,.,1746.0
1,AUS,1275.0,4.0,2951.0
2,AUT,1619.0,375.0,5066.0
3,BEL,399.0,.,1799.0
4,BRA,22505.0,17940.0,10666.0


In [304]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

,country,wood_pulp_production_2021,wood_pulp_market_pulps_2021,paper_paperboard_production_2021
0,ARG,609.0,NaN,1746.0
1,AUS,1275.0,4.0,2951.0
2,AUT,1619.0,375.0,5066.0
3,BEL,399.0,NaN,1799.0
4,BRA,22505.0,17940.0,10666.0


In [305]:
data.to_csv('data/cleaned/PULP_AND_PAPER_PRODUCTION_BY_COUNTRY_2021.csv', index=False)

# `UTILIZATION OF RECOVERED PAPER BY COUNTRY 2020.xls`

In [306]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/UTILIZATION OF RECOVERED PAPER BY COUNTRY 2020.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

recovered_paper_totals = [sheet.cell_value(rowx=16, colx=1) for sheet in sheets]
mixed_grades = [sheet.cell_value(rowx=16, colx=2) for sheet in sheets]
corrugated_solid_containers = [sheet.cell_value(rowx=16, colx=3) for sheet in sheets]
newspapers = [sheet.cell_value(rowx=16, colx=4) for sheet in sheets]
wood_frees = [sheet.cell_value(rowx=16, colx=5) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'recovered_paper_totals_2020': recovered_paper_totals,
    'mixed_grades_2020': mixed_grades,
    'corrugated_solid_containers_2020' : corrugated_solid_containers,
    'newspapers_2020' : newspapers,
    'wood_free_2020' : wood_frees
})

print(f"Shape is: {data.shape}")
data.head()

Shape is: (28, 6)


,country,recovered_paper_totals_2020,mixed_grades_2020,corrugated_solid_containers_2020,newspapers_2020,wood_free_2020
0,ARG,909.0,.,.,.,.
1,AUS,1808.0,60.0,1076.0,50.0,622.0
2,AUT,2558.0,379.0,1274.0,547.0,358.0
3,BLR,258.0,.,.,.,.
4,BEL,1221.0,39.0,538.0,601.0,43.0


In [308]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

,country,recovered_paper_totals_2020,mixed_grades_2020,corrugated_solid_containers_2020,newspapers_2020,wood_free_2020
0,ARG,909.0,NaN,NaN,NaN,NaN
1,AUS,1808.0,60.0,1076.0,50.0,622.0
2,AUT,2558.0,379.0,1274.0,547.0,358.0
3,BLR,258.0,NaN,NaN,NaN,NaN
4,BEL,1221.0,39.0,538.0,601.0,43.0


In [310]:
data.to_csv('data/cleaned/UTILIZATION OF RECOVERED PAPER BY COUNTRY 2020.csv', index=False)

# `UTILIZATION_OF_RECOVERED_PAPER_BY_COUNTRY_2021.xls`

In [313]:
# Open the xls file
workbook = xlrd.open_workbook('data/raw/UTILIZATION_OF_RECOVERED_PAPER_BY_COUNTRY_2021.xls')

# Get a list of all sheet names in the xls file
sheet_names = workbook.sheet_names()

# Filter the sheet names to get only those with 3 characters
filtered_sheet_names = [name for name in sheet_names if len(name) == 3]

# Load each filtered sheet into a list of lists
sheets = [workbook.sheet_by_name(name) for name in filtered_sheet_names]

recovered_paper_totals = [sheet.cell_value(rowx=16, colx=1) for sheet in sheets]
mixed_grades = [sheet.cell_value(rowx=16, colx=2) for sheet in sheets]
corrugated_solid_containers = [sheet.cell_value(rowx=16, colx=3) for sheet in sheets]
newspapers = [sheet.cell_value(rowx=16, colx=4) for sheet in sheets]
wood_frees = [sheet.cell_value(rowx=16, colx=5) for sheet in sheets]

data = pd.DataFrame({
    'country': filtered_sheet_names,
    'recovered_paper_totals_2021': recovered_paper_totals,
    'mixed_grades_2021': mixed_grades,
    'corrugated_solid_containers_2021' : corrugated_solid_containers,
    'newspapers_2021' : newspapers,
    'wood_free_2021' : wood_frees
})

print(f"Shape is: {data.shape}")
data.head()

Shape is: (23, 6)


,country,recovered_paper_totals_2021,mixed_grades_2021,corrugated_solid_containers_2021,newspapers_2021,wood_free_2021
0,ARG,1032.0,.,.,.,.
1,AUS,1732.0,52.0,1084.0,15.0,581.0
2,AUT,2545.0,317.0,1343.0,554.0,331.0
3,BEL,1179.0,40.0,540.0,567.0,32.0
4,BRA,4890.0,192.0,3990.0,95.0,613.0


In [314]:
def transform_col(cell):
    if cell == '.':
        return None
    else:
        return cell

for col in data.columns:
    if col != 'country':
        data[col] = data[col].apply(transform_col)

data.head()

,country,recovered_paper_totals_2021,mixed_grades_2021,corrugated_solid_containers_2021,newspapers_2021,wood_free_2021
0,ARG,1032.0,NaN,NaN,NaN,NaN
1,AUS,1732.0,52.0,1084.0,15.0,581.0
2,AUT,2545.0,317.0,1343.0,554.0,331.0
3,BEL,1179.0,40.0,540.0,567.0,32.0
4,BRA,4890.0,192.0,3990.0,95.0,613.0


In [315]:
data.to_csv('data/cleaned/UTILIZATION_OF_RECOVERED_PAPER_BY_COUNTRY_2021.csv', index=False)